# Single Neuron Biophysics notebooks

*David Sterratt, October 2025*

These notebooks accompany the MSc Neuroscience lecture on Single Neuron Biophysics

## Part 3: Using the NEURON simulation library to simulate the Hodgkin-Huxley model

We'll now use NEURON to simulate the Hodgkin-Huxely model.

In [ ]:
!pip install neuron

Now import the relevant parts of NEURON. Don't worry about the warning about the DISPLAY variable.

In [ ]:
from neuron import n
from neuron.units import ms, mV
n.load_file("stdrun.hoc")

We'll also import the graphics plotting package before we start

In [ ]:
from bokeh.io import output_notebook # A module that we need to plot 
import bokeh.plotting as plt         # A module that we need to plot 
output_notebook()

### 3.A Create and run a cell model

We will first construct a NEURON simulation based on the Hodgkin-Huxley model of the action
potential in a small, isopotential section of the squid giant axon.+

![img](hh-compartmentA.png)

To create the Hodgkin-Huxley model, we use similar code to the previous tutorial, but we insert an `hh` mechanism instead of a `pas` mechanism.

In [ ]:
# Create the soma
soma = n.Section("soma")

# Insert the HH mechanism
soma.insert(n.hh)

## Print the section
soma.psection()

The table below shows the mathematical symbols and their equivalents in NEURON. Some mathematical relationships
are also shown.

![img](hh-symbols-table.png)

We can now run the simulation with the code below. What would you expect to see?

In [ ]:
# Set up recordings of the membrane potential and time
v = n.Vector().record(soma(0.5)._ref_v)      # Membrane potential vector
i_na = n.Vector().record(soma(0.5)._ref_ina)  # Current through passive mechanism
i_k = n.Vector().record(soma(0.5)._ref_ik)  # Current through passive mechanism
t = n.Vector().record(n._ref_t)              # Time stamp vector

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(100 * ms)

In [ ]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)


There are no input currents to the neuron at this point. The phenomenon you observe here suggests
it’s a good idea to give model neurons a while to settle down before injecting current.

### 3.B Current clamp

We'll now insert a current  clamp (as with the passive cell in the previous part) and run the model.

In [ ]:
# Insert a current clamp electrode in the centre of the soma
stim = n.IClamp(soma(0.5))
stim.amp   =  2 # amplitude in nA
stim.delay = 50 # delay in ms
stim.dur   =  1 # duration in ms

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(100 * ms)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)

Now run the simulation 8 times, increasing the amplitude by 2 nA each time. 

- What do you see?
- What is the lowest amplitude current required to produce an action potential?

### 3.C Blocking conductances

For the next part of the practical, the current should be big enough to elicit an action potential. How-
ever, to make it clear what happens, it shouldn’t be too big; 15 nA is a suitable value to set the current
to.

Hodgkin and Huxley (1952) had to change the concentration of sodium and potassium in order to
see how each affected the action potential. However, in the 1960s, methods for blocking channels
pharmacologically were developed. Tetrodotoxin (TTX) is isolated from the Japanese puffer fish *fugu*
and blocks Na+ channels involved in generating action potentials. Tetraethylammonium (TEA) blocks
K+ channels.

The effect of blocking one or other conductance can easily be tested with this simulation.

In [ ]:
# Block the potassium channel
soma(0.5).hh.gkbar = 0

# Stimulus
stim = n.IClamp(soma(0.5))
stim.amp = 15    # amplitude in nA - should produce an action potential
stim.delay = 50  # delay in ms
stim.dur = 1     # duration in ms

# Run the simulation again

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(100 * ms)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)

- What do you see?
- What can you deduce from this?

Now unblock the potassium channels by setting them back to their original value, and try blocking the sodium channels:

In [ ]:
# Unblock the potassium channel
soma(0.5).hh.gkbar = 0.036

# Block the sodium channel 
soma(0.5).hh.gnabar = 0

# Run the simulation again

# Set the initial membrane potential to -65mV
n.finitialize(-65 * mV)

# Run for 40 milliseconds
n.continuerun(100 * ms)

f = plt.figure(x_axis_label="t (ms)", y_axis_label="v (mV)", aspect_ratio=2)
f.line(t, v, line_width=2)
plt.show(f)

- What do you see? (Note the scale!)
- What can you deduce from this?


Finally let's unblock the sodium channels.

In [ ]:
# Block the sodium channel 
soma(0.5).hh.gnabar = 0.12

- What do you see? (Note the scale!)
- What can you deduce from this?